# Construct Inputs from Reanalysis products
* This iteration takes a lat/lon box and reanalysis product to force with. For now will be RYF so that Angus' code still applies. 
* Also takes bathymetry and generates a hgrid based on user input and the bathy. Everything then interpolated onto this hgrid like before. 
* This notebook should be a testbed, then all the bits for constructing the domain turned into functions

In [1]:
import netCDF4
import xarray as xr
import xesmf as xe
from itertools import cycle
import os
import dask
import numpy as np
import pandas as pd
import dask.array as da
import dask.bag as db
from pykdtree.kdtree import KDTree
from dask.diagnostics import ProgressBar
import subprocess

import matplotlib.pyplot as plt
import motuclient
os.chdir("cosima_regional/mom6-regional-scripts")
print(os.getcwd())
import datetime as dt

# Import scripts
from regional_model_scripts import input_datasets, interp_segment,prepare_segments, time_rotate,regrid_runoff,sel_hgrid_indices
from boundary_tides import write_tpxo

om2inputdir = "/g/data/ik11/inputs/access-om2/input_08022019/mom_01deg/" ## Access om2_01 input for topography and hgrid
initinputdir = "/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output1077" ## Access om2_01 output for BCs and IC
toolinputdir = "/home/157/ahg157/repos/mom5/src/tools/" ## Compiled tools needed for construction of mask tables
    
xr.set_options(keep_attrs=True) ## This ensures that performing simple operations with xarray (eg converting temperature to Celsius) preserves attributes

from dask.distributed import Client
client = Client()
client

/home/149/ab8992/cosima_regional/mom6-regional-scripts


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35439,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:33091,Total threads: 2
Dashboard: /proxy/46175/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:33527,


In [2]:

## Choose your coordinates and the name of your experiment
yextent = [-48,-38.95]
xextent = [-217 , -210]
expt_name = "reanalysis-small"

pwd = "$KQ%QqFxjSSbE2"
usr = "abarnes"


daterange = ["2015-01-01 00:00:00", "2015-01-31 00:00:00"]


## Place where all your input files go
inputdir = f"/scratch/v45/ab8992/mom6/regional_configs/{expt_name}/"

## Directory where you'll run the experiment from
rundir = f"/home/149/ab8992/mom6_rundirs/{expt_name}/"

## Directory where raw downloads go before processing
tmpdir = f"/scratch/v45/ab8992/reanalysis_tmp/{expt_name}/"

for i in [inputdir,rundir,tmpdir]:
    try:
        os.mkdir(i)
    except:
        pass

if "temp" not in os.listdir(inputdir):
    os.mkdir(inputdir + "temp")
if "weights" not in os.listdir(inputdir):
    os.mkdir(inputdir + "weights")
if "forcing" not in os.listdir(inputdir):
    os.mkdir(inputdir + "forcing")

    


## Firstly, generate a bash request that downloads the IC anc BC from the GLORYs reanalysis product 

In [19]:
import regional_model_scripts as ml 


file = open(f"{tmpdir}/get_oceanfiles.sh","w")
file.write(
        moturequests(xextent, yextent, daterange, tmpdir, usr, pwd,["north","south","east","west"])
)
file.close()

ml.motu_requests(xextent, yextent, daterange, tmpdir, usr, pwd,["north","south","east","west"])


"#!/bin/bash\n\n\nprintf 'processing north segment' \npython -m motuclient --motu https://my.cmems-du.eu/motu-web/Motu --service-id GLOBAL_MULTIYEAR_PHY_001_030-TDS --product-id cmems_mod_glo_phy_my_0.083_P1D-m --longitude-min -217 --longitude-max -210 --latitude-min -39.25 --latitude-max -38.650000000000006 --date-min 2015-01-01 00:00:00 --date-max 2015-01-31 00:00:00 --depth-min 0.49 --depth-max 6000 --variable so --variable thetao --variable vo --variable zos --variable uo --out-dir /scratch/v45/ab8992/reanalysis_tmp/reanalysis-small --out-name north_segment_unprocessed --user 'abarnes' --pwd '$KQ%QqFxjSSbE2' & \n\nprintf 'processing south segment' \npython -m motuclient --motu https://my.cmems-du.eu/motu-web/Motu --service-id GLOBAL_MULTIYEAR_PHY_001_030-TDS --product-id cmems_mod_glo_phy_my_0.083_P1D-m --longitude-min -217 --longitude-max -210 --latitude-min -48.3 --latitude-max -47.7 --date-min 2015-01-01 00:00:00 --date-max 2015-01-31 00:00:00 --depth-min 0.49 --depth-max 6000 -

## Read in our saved boundary segments

In [3]:
north = xr.open_dataset("/scratch/v45/ab8992/reanalysis_tmp/reanalysis-small/north_segment_unprocessed",decode_times=False)
north

<xarray.Dataset>
Dimensions:    (depth: 50, latitude: 8, time: 31, longitude: 85)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * latitude   (latitude) float32 -39.25 -39.17 -39.08 ... -38.83 -38.75 -38.67
  * time       (time) float64 5.698e+05 5.698e+05 ... 5.705e+05 5.705e+05
  * longitude  (longitude) float32 143.0 143.1 143.2 143.2 ... 149.8 149.9 150.0
Data variables:
    vo         (time, depth, latitude, longitude) float32 ...
    thetao     (time, depth, latitude, longitude) float32 ...
    uo         (time, depth, latitude, longitude) float32 ...
    so         (time, depth, latitude, longitude) float32 ...
    zos        (time, latitude, longitude) float32 ...
Attributes: (12/25)
    title:                              daily mean fields from Global Ocean P...
    easting:                            longitude
    northing:                           latitude
    history:                            2022/05/25 21:54:07 MERCATOR OCEAN Ne...
    source:                             MERCATOR GLORYS12V1
    institution:                        MERCATOR OCEAN
    ...                                 ...
    FROM_ORIGINAL_FILE__longitude_max:  179.91667
    FROM_ORIGINAL_FILE__latitude_min:   -80.0
    FROM_ORIGINAL_FILE__latitude_max:   90.0
    z_min:                              0.494025
    z_max:                              5727.917
    _CoordSysBuilder:                   ucar.nc2.dataset.conv.CF1Convention

## Make a class for each boundary segment. 
* Requires inputdir to raw segment, the hgrid, the segment cardinal direction and names of variables
* Has a re-written brushcutter function as a method. Might still need to save a zarr file, but chunking should be easier given rectangular slices 
* Overarching function should take care of the hgrid and initial condition?

In [61]:
toolpath = "/home/157/ahg157/repos/mom5/src/tools/" ## Compiled tools needed for construction of mask tables


class segment:
    """
    Class to turn raw boundary segment data into MOM6 boundary segments. 
    """
    def __init__(self, hgrid,infile, outfolder,varnames,seg_name,orientation, gridtype="A",vcoord_type = "height"):
        """
        hgrid:        xarray        the horizontal grid used for domain
        infolder:     string        path to the raw, unprocessed boundary segment
        outfolder:    string        path to folder where the model inputs will be stored
        varnames:     dictionary    Mapping between the variable / dimension names and standard naming convension of this pipeline. eg {"xq":"longitude,"yh":"latitude","salt":"salinity...}
        orientation:  string        Cardinal direction (lowercase) of the boundary segment
        gridtype:     string        A,B or C type grid
        seg_name:     string        Name of the segment. Something like 'segment_001'
        vcoord_type:  string        Vertical coordinate is either interfacial 'height' or layer 'thickness'. Handles appropriately
        """

        ## Store coordinate names
        if gridtype == "A":
            self.x = varnames["x"]
            self.y = varnames["y"]
            
            ## REGRID A to C

        elif gridtype in ("B","C"):
            self.xq = varnames["xq"]
            self.xh = varnames["xh"]
            self.yq = varnames["yq"]
            self.yh = varnames["yh"]

        ## Store variable names 
        self.temp = varnames["temp"]
        self.u = varnames["u"]
        self.v = varnames["v"]
        self.salt = varnames["salt"]
        self.time = varnames["time"]
        self.eta = varnames["eta"]
        self.z = varnames["z"]

        ## Store other data
        self.infile = infile
        self.outfolder = outfolder
        self.orientation = orientation.lower() ## might not be needed? NSEW
        self.grid = gridtype
        self.hgrid = hgrid
        self.seg_name = seg_name
        self.vcoord_type = vcoord_type
    def brushcut(self):
        ### Implement brushcutter scheme on single segment ### 
        # print(self.infile + f"/{self.orientation}_segment_unprocessed")
        rawseg = xr.open_dataset(self.infile)

        ## Depending on the orientation of the segment, cut out the right bit of the hgrid 
        ## and define which coordinate is along or into the segment
        if self.orientation == "north":
            hgrid_seg = self.hgrid.isel(nyp = [-1])
            perpendicular = "ny"
            parallel = "nx"

        if self.orientation == "south":
            hgrid_seg = self.hgrid.isel(nyp = [0])
            perpendicular = "ny"
            parallel = "nx"
    

        if self.orientation == "east":
            hgrid_seg = self.hgrid.isel(nxp = [-1])
            perpendicular = "nx"
            parallel = "ny"

        if self.orientation == "west":
            hgrid_seg = self.hgrid.isel(nxp = [0])
            perpendicular = "nx"
            parallel = "ny"

        ## Need to keep track of which axis the 'main' coordinate corresponds to for later on when re-adding the 'secondary' axis
        if perpendicular == "ny":
            axis1 = 3
            axis2 = 2
        else:
            axis1 = 2
            axis2 = 3

        

        ## Grid for interpolating our fields
        interp_grid = xr.Dataset(
            {
                "lat": ([f"{parallel}_{self.seg_name}"], hgrid_seg.y.squeeze().data),
                "lon": ([f"{parallel}_{self.seg_name}"], hgrid_seg.x.squeeze().data),
            }
        ).set_coords(["lat","lon"])


        if self.grid == "A":
            ## In this case velocities and tracers all on same points
            regridder = xe.Regridder(
                rawseg[self.u].rename({self.x:"lon", self.y:"lat"}),
                interp_grid,
                "bilinear",
                locstream_out=True,
                reuse_weights=False,
                filename=self.outfolder + f"weights/bilinear_velocity_weights_{self.orientation}.nc",
            )
            segment_out = xr.merge([
                regridder(rawseg[
                [self.u,self.v,self.salt,self.temp,self.eta]
                ])])
            

        if self.grid =="B":
            ## All tracers on one grid, all velocities on another
            regridder_velocity = xe.Regridder(
                rawseg[self.u].rename({self.xq:"lon", self.yq:"lat"}),
                interp_grid,
                "bilinear",
                locstream_out=True,
                reuse_weights=False,
                filename= self.outfolder,
            )

            regridder_tracer = xe.Regridder(
                rawseg[self.salt].rename({self.xh:"lon", self.yh:"lat"}),
                interp_grid,
                "bilinear",
                locstream_out=True,
                reuse_weights=False,
                filename=self.outfolder + f"weights/bilinear_velocity_weights_{self.orientation}.nc",
            )


            segment_out = xr.merge([
                regridder_velocity(rawseg[[self.u,self.v]]), 
                regridder_tracer(rawseg[[self.salt,self.temp,self.eta]]), 
                 ])
            
        if self.grid =="C":
            ## All tracers on one grid, all velocities on another
            regridder_uvelocity = xe.Regridder(
                rawseg[self.u].rename({self.xq:"lon", self.yh:"lat"}),
                interp_grid,
                "bilinear",
                locstream_out=True,
                reuse_weights=False,
                filename= self.outfolder,
            )

            regridder_vvelocity = xe.Regridder(
                rawseg[self.v].rename({self.xh:"lon", self.yq:"lat"}),
                interp_grid,
                "bilinear",
                locstream_out=True,
                reuse_weights=False,
                filename= self.outfolder,
            )

            regridder_tracer = xe.Regridder(
                rawseg[self.salt].rename({self.xh:"lon", self.yh:"lat"}),
                interp_grid,
                "bilinear",
                locstream_out=True,
                reuse_weights=False,
                filename=self.outfolder + f"weights/bilinear_velocity_weights_{self.orientation}.nc",
            )


            segment_out = xr.merge([
                regridder_vvelocity(rawseg[[self.v]]), 
                regridder_uvelocity(rawseg[[self.u]]), 
                regridder_tracer(rawseg[[self.salt,self.temp,self.eta]]), 
                 ])

        ## segment out now contains our interpolated boundary.
        ## Now, we need to fix up all the metadata and save
        
        del segment_out["lon"]
        del segment_out["lat"]
        ## Convert temperatures to celsius # use pint
        if np.min(segment_out[self.temp].isel({self.time:0})) > 100:
            segment_out[self.temp] -= 273.15

        # fill in NaNs
        segment_out = (
            segment_out
            .ffill(self.z)
            .interpolate_na(f"{parallel}_{self.seg_name}")
            .ffill(f"{parallel}_{self.seg_name}")
            .bfill(f"{parallel}_{self.seg_name}")
        )


        ##### FIX UP COORDINATE METADATA #####


        # Dictionary we built for encoding the netcdf at end
        encoding_dict = {
            "time": {
                "dtype": "double",
                "units": "days since 1900-01-01 12:00:00",
                "calendar": "noleap",
            },
            f"nx_{self.seg_name}": {
                "dtype": "int32",
            },
            f"ny_{self.seg_name}": {
                "dtype": "int32",
            },
        }


        ### Generate our dz variable. This needs to be in layer thicknesses
        if self.vcoord_type == "height":
            dz = segment_out[self.z].diff(self.z)
            dz.name = "dz"
            dz = xr.concat([dz,dz[-1]],dim = self.z)

        else:
            dz = segment_out[self.z]
            dz.name = "dz"
        del segment_out[self.z]


        # Here, keep in mind that 'var' keeps track of the mom6 variable names we want, and self.var / getattr(self,var) will return the name of the variable from the original data
        for var in ["salt","temp","u","v"]: ## Replace with more generic list of tracer variables that might be included?
            v = f"{var}_{self.seg_name}"
            ## Rename each variable in dataset
            segment_out = segment_out.rename({getattr(self,var): v})

            ## Rename vertical coordinate for this variable
            segment_out[f"{var}_{self.seg_name}"] = segment_out[f"{var}_{self.seg_name}"].rename({self.z: f"nz_{self.seg_name}_{var}"})

            ## Replace the old depth coordinates with incremental integers 
            segment_out[f"nz_{self.seg_name}_{var}"] = np.arange(segment_out[f"nz_{self.seg_name}_{var}"].size)

            ## Re-add the secondary dimension (even though it represents one value..)
            segment_out[v] = segment_out[v].expand_dims(
                f"{perpendicular}_{self.seg_name}",axis = axis2
            )


            print(segment_out[v].shape)
            ## Add the layer thicknesses
            segment_out[f"dz_{v}"] = (
                ["time", f"nz_{v}", f"ny_{self.seg_name}", f"nx_{self.seg_name}"],
                da.broadcast_to(
                    dz.data[None, :, None, None],
                    segment_out[v].shape,
                    chunks=(1, None, None, None),
                ),
            )


            encoding_dict[v] = {
                "_FillValue": netCDF4.default_fillvals["f8"],
                "zlib": True,
                # "chunksizes": tuple(s),
            }
            encoding_dict[f"dz_{v}"] = {
                "_FillValue": netCDF4.default_fillvals["f8"],
                "zlib": True,
                # "chunksizes": tuple(s),
            }

            ## appears to be another variable just with integers??
            encoding_dict[f"nz_{self.seg_name}_{var}"] = {
                "dtype": "int32"
            }

        ## Treat eta separately since it has no vertical coordinate. Do the same things as for the surface variables above
        segment_out = segment_out.rename({self.eta: f"eta_{self.seg_name}"})
        encoding_dict[ f"eta_{self.seg_name}"] = {
            "_FillValue": netCDF4.default_fillvals["f8"],
        }
        segment_out[f"eta_{self.seg_name}"] = segment_out[f"eta_{self.seg_name}"].expand_dims(
            f"{perpendicular}_{self.seg_name}",axis = axis2 - 1
        )

        # Overwrite the actual lat/lon values in the dimensions, replace with incrementing integers
        segment_out[f"{parallel}_{self.seg_name}"] = np.arange(segment_out[f"{parallel}_{self.seg_name}"].size)
        segment_out[f"{perpendicular}_{self.seg_name}"] = [0]

        # Store actual lat/lon values here as variables rather than coordinates
        segment_out[f"lon_{self.seg_name}"] = ([f"ny_{self.seg_name}", f"nx_{self.seg_name}"], hgrid_seg.x.data)
        segment_out[f"lat_{self.seg_name}"] = ([f"ny_{self.seg_name}", f"nx_{self.seg_name}"], hgrid_seg.y.data)


        with ProgressBar():
            segment_out["time"] = segment_out["time"].assign_attrs({"modulo":" "}) ## Add modulo attribute for MOM6 to treat as repeat forcing
            segment_out.load().to_netcdf(
                self.outfolder + f"forcing/forcing_obc_{self.seg_name}.nc", encoding=encoding_dict, unlimited_dims="time"
                )

        return segment_out , encoding_dict
    



## Generate the hgrid based on resolution and horizontal extents only

In [76]:

res = 0.1

## Total number of qpoints in x
qpoints_x = int((xextent[1] - xextent[0])/res) + 1
qpoints_y = int((yextent[1] - yextent[0])/res) + 1
if qpoints_x == 0 or qpoints_y == 0:
    raise ValueError("Please ensure domain extents match chosen resolution")
Xq = np.linspace(xextent[0],xextent[1],qpoints_x)
Yq = np.linspace(yextent[0],yextent[1],qpoints_y)

Xt = np.linspace(
    xextent[0] + res/2,
    xextent[1] - res/2,
    qpoints_x - 1
    )

Yt = np.Xt = np.linspace(
    yextent[0] + res/2,
    yextent[1] - res/2,
    qpoints_y - 1
    )

# broadcast to meshgrid
Xt, Yt = np.meshgrid(Xt, Yt)
Xq, Yq = np.meshgrid(Xq, Yq)

# create output dataset
ds = xr.Dataset({
    "grid_lon": (['grid_yc', 'grid_xc'], Xq),
    'grid_lat': (['grid_yc', 'grid_xc'], Yq),
    'grid_lont': (['grid_yt', 'grid_xt'], Xt),
    'grid_latt': (['grid_yt', 'grid_xt'], Yt),
})
ds.to_netcdf(inputdir + "/grid.nc")

## Generate the hgrid with fretools. Need to generalise later to not rely on random scripts!
args = "--grid_type from_file --my_grid_file grid.nc".split(" ")
subprocess.run([toolpath + "make_hgrid/make_hgrid"] + args, cwd=inputdir)
subprocess.run(["mv","horizontal_grid.nc","hgrid.nc"],cwd=inputdir)


CompletedProcess(args=['mv', 'horizontal_grid.nc', 'hgrid.nc'], returncode=0)

## Construct brushcutter draft for segment



In [62]:
hgrid = xr.open_dataset(inputdir + "/hgrid.nc")
rawseg = xr.open_dataset(tmpdir + "/north_segment_unprocessed")

orientation = "north"

seg = segment(hgrid,tmpdir + "/north_segment_unprocessed",inputdir,{"time":"time","y":"latitude","x":"longitude","z":"depth","eta":"zos","salt":"so","temp":"thetao","u":"uo","v":"vo"},"segment_001","north")
out , encoding = seg.brushcut()


/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xesmf/frontend.py:597: UserWarning: Using dimensions ('latitude', 'longitude') from data variable uo as the horizontal dimensions for the regridding.
  warnings.warn(


<xarray.Dataset>
Dimensions:  (time: 31, depth: 50, nx_segment_001: 141)
Coordinates:
  * time     (time) datetime64[ns] 2015-01-01T12:00:00 ... 2015-01-31T12:00:00
Dimensions without coordinates: depth, nx_segment_001
Data variables:
    uo       (time, depth, nx_segment_001) float32 -0.00232 ... -0.01538
    vo       (time, depth, nx_segment_001) float32 -0.05847 -0.0439 ... 0.005249
    so       (time, depth, nx_segment_001) float32 35.7 35.7 ... 34.72 34.72
    thetao   (time, depth, nx_segment_001) float32 17.32 17.33 ... 0.7898 0.767
    zos      (time, nx_segment_001) float32 0.1012 0.1031 ... 0.1268 0.1421
Attributes:
    regrid_method:  bilinear
<xarray.DataArray 'dz' (depth: 50)>
array([  1.04735  ,   1.104294 ,   1.173826 ,   1.2587292,   1.36239  ,
         1.488946 ,   1.6434369,   1.8320026,   2.0621405,   2.3429298,
         2.6854897,   3.10326  ,   3.6125908,   4.23332  ,   4.98942  ,
         5.9099007,   7.02964  ,   8.390598 ,  10.042976 ,  12.046585 ,
        14.47

In [33]:
encoding

{'time': {'dtype': 'double',
  'units': 'days since 1900-01-01 12:00:00',
  'calendar': 'noleap'},
 'nx_segment_001': {'dtype': 'int32'},
 'ny_segment_001': {'dtype': 'int32'},
 'salt_segment_001': {'_FillValue': 9.969209968386869e+36, 'zlib': True},
 'dz_salt_segment_001': {'_FillValue': 9.969209968386869e+36, 'zlib': True},
 'nz_segment_001_salt': {'dtype': 'int32'},
 'temp_segment_001': {'_FillValue': 9.969209968386869e+36, 'zlib': True},
 'dz_temp_segment_001': {'_FillValue': 9.969209968386869e+36, 'zlib': True},
 'nz_segment_001_temp': {'dtype': 'int32'},
 'u_segment_001': {'_FillValue': 9.969209968386869e+36, 'zlib': True},
 'dz_u_segment_001': {'_FillValue': 9.969209968386869e+36, 'zlib': True},
 'nz_segment_001_u': {'dtype': 'int32'},
 'v_segment_001': {'_FillValue': 9.969209968386869e+36, 'zlib': True},
 'dz_v_segment_001': {'_FillValue': 9.969209968386869e+36, 'zlib': True},
 'nz_segment_001_v': {'dtype': 'int32'},
 'eta_segment_001': {'_FillValue': 9.969209968386869e+36}}

In [28]:
out

<xarray.Dataset>
Dimensions:              (time: 31, nz_segment_001_u: 50, xh_segment_001: 1,
                          yh_segment_001: 141, nz_segment_001_v: 50,
                          nz_segment_001_salt: 50, nz_segment_001_temp: 50,
                          nz_salt_segment_001: 50, ny_segment_001: 1,
                          nx_segment_001: 141, nz_temp_segment_001: 50,
                          nz_u_segment_001: 50, nz_v_segment_001: 50)
Coordinates:
  * time                 (time) datetime64[ns] 2015-01-01T12:00:00 ... 2015-0...
  * nz_segment_001_salt  (nz_segment_001_salt) int64 0 1 2 3 4 ... 46 47 48 49
  * nz_segment_001_temp  (nz_segment_001_temp) int64 0 1 2 3 4 ... 46 47 48 49
  * nz_segment_001_u     (nz_segment_001_u) int64 0 1 2 3 4 5 ... 45 46 47 48 49
  * nz_segment_001_v     (nz_segment_001_v) int64 0 1 2 3 4 5 ... 45 46 47 48 49
  * yh_segment_001       (yh_segment_001) int64 0 1 2 3 4 ... 137 138 139 140
  * xh_segment_001       (xh_segment_001) int64 0
Dimensions without coordinates: nz_salt_segment_001, ny_segment_001,
                                nx_segment_001, nz_temp_segment_001,
                                nz_u_segment_001, nz_v_segment_001
Data variables:
    u_segment_001        (time, nz_segment_001_u, xh_segment_001, yh_segment_001) float32 ...
    v_segment_001        (time, nz_segment_001_v, xh_segment_001, yh_segment_001) float32 ...
    salt_segment_001     (time, nz_segment_001_salt, xh_segment_001, yh_segment_001) float32 ...
    temp_segment_001     (time, nz_segment_001_temp, xh_segment_001, yh_segment_001) float32 ...
    eta_segment_001      (time, xh_segment_001, yh_segment_001) float32 0.101...
    dz_salt_segment_001  (time, nz_salt_segment_001, ny_segment_001, nx_segment_001) float32 dask.array<chunksize=(1, 50, 1, 141), meta=np.ndarray>
    dz_temp_segment_001  (time, nz_temp_segment_001, ny_segment_001, nx_segment_001) float32 dask.array<chunksize=(1, 50, 1, 141), meta=np.ndarray>
    dz_u_segment_001     (time, nz_u_segment_001, ny_segment_001, nx_segment_001) float32 dask.array<chunksize=(1, 50, 1, 141), meta=np.ndarray>
    dz_v_segment_001     (time, nz_v_segment_001, ny_segment_001, nx_segment_001) float32 dask.array<chunksize=(1, 50, 1, 141), meta=np.ndarray>
    lon_segment_001      (ny_segment_001, nx_segment_001) float64 -217.0 ... ...
    lat_segment_001      (ny_segment_001, nx_segment_001) float64 -38.95 ... ...
Attributes:
    regrid_method:  bilinear

In [37]:
out

2023-04-19 17:22:56,930 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2023-04-19 17:22:57,336 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-04-19 17:22:58,650 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-04-19 17:22:59,052 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-04-19 17:23:00,300 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2023-04-19 17:23:00,679 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2023-04-19 17:23:01,888 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
2023-04-19 17:23:02,277 - distributed.utils_perf - WARNING - full garbage collections took

<xarray.Dataset>
Dimensions:              (time: 31, nz_segment_001_u: 50, xh_segment_001: 1,
                          yh_segment_001: 141, nz_segment_001_v: 50,
                          nz_segment_001_salt: 50, nz_segment_001_temp: 50,
                          nz_salt_segment_001: 50, ny_segment_001: 1,
                          nx_segment_001: 141, nz_temp_segment_001: 50,
                          nz_u_segment_001: 50, nz_v_segment_001: 50)
Coordinates:
  * time                 (time) datetime64[ns] 2015-01-01T12:00:00 ... 2015-0...
  * nz_segment_001_salt  (nz_segment_001_salt) int64 0 1 2 3 4 ... 46 47 48 49
  * nz_segment_001_temp  (nz_segment_001_temp) int64 0 1 2 3 4 ... 46 47 48 49
  * nz_segment_001_u     (nz_segment_001_u) int64 0 1 2 3 4 5 ... 45 46 47 48 49
  * nz_segment_001_v     (nz_segment_001_v) int64 0 1 2 3 4 5 ... 45 46 47 48 49
  * yh_segment_001       (yh_segment_001) int64 0 1 2 3 4 ... 137 138 139 140
  * xh_segment_001       (xh_segment_001) int64 0
Dimensions without coordinates: nz_salt_segment_001, ny_segment_001,
                                nx_segment_001, nz_temp_segment_001,
                                nz_u_segment_001, nz_v_segment_001
Data variables:
    u_segment_001        (time, nz_segment_001_u, xh_segment_001, yh_segment_001) float32 ...
    v_segment_001        (time, nz_segment_001_v, xh_segment_001, yh_segment_001) float32 ...
    salt_segment_001     (time, nz_segment_001_salt, xh_segment_001, yh_segment_001) float32 ...
    temp_segment_001     (time, nz_segment_001_temp, xh_segment_001, yh_segment_001) float32 ...
    eta_segment_001      (time, xh_segment_001, yh_segment_001) float32 0.101...
    dz_salt_segment_001  (time, nz_salt_segment_001, ny_segment_001, nx_segment_001) float32 ...
    dz_temp_segment_001  (time, nz_temp_segment_001, ny_segment_001, nx_segment_001) float32 ...
    dz_u_segment_001     (time, nz_u_segment_001, ny_segment_001, nx_segment_001) float32 ...
    dz_v_segment_001     (time, nz_v_segment_001, ny_segment_001, nx_segment_001) float32 ...
    lon_segment_001      (ny_segment_001, nx_segment_001) float64 -217.0 ... ...
    lat_segment_001      (ny_segment_001, nx_segment_001) float64 -38.95 ... ...
Attributes:
    regrid_method:  bilinear

In [63]:
out.load().to_netcdf(
    inputdir + f"forcing/forcing_obc_segment_001.nc", encoding=encoding, unlimited_dims="time"
    )

In [79]:
regridder_tracer = xe.Regridder(
    raw,
    interp,
    "bilinear",
    locstream_out=True,
    reuse_weights=False,
    # filename=self.outfolder + f"weights/bilinear_velocity_weights_{self.orientation}.nc",
)

regridder_tracer(raw)

<xarray.DataArray (time: 31, depth: 50, location: 141)>
array([[[35.699028, 35.703056, 35.70794 , 35.71453 , 35.722588,
         35.731377, 35.734676, 35.736015, 35.73345 , 35.729546,
         35.724968, 35.728996, 35.735283, 35.746086, 35.75603 ,
         35.765556, 35.7694  , 35.772514, 35.774162, 35.776054,
         35.77807 , 35.78118 , 35.78588 , 35.793755, 35.8059  ,
         35.820187, 35.828976, 35.836422, 35.841183, 35.8457  ,
         35.850094, 35.85577 , 35.861263, 35.86639 , 35.870174,
         35.873287, 35.874935, 35.876705, 35.87872 , 35.881466,
         35.88458 , 35.88641 , 35.887386, 35.886654, 35.885677,
         35.88458 , 35.886227, 35.887997, 35.89001 , 35.889828,
         35.888546, 35.89056 , 35.893185, 35.89703 , 35.895382,
         35.890987, 35.913513, 35.93616 , 35.959045, 35.961796,
         35.954468, 35.961613, 35.97272 , 35.99176 , 36.004456,
         36.013977,       nan,       nan,       nan, 36.013306,
         36.002686, 35.999756,       nan,       nan,       nan,
               nan, 35.993652, 35.992493, 35.99121 , 35.989685,
         35.988037, 35.980347, 35.967407, 35.943974, 35.928593,
         35.917233, 35.900208, 35.88208 , 35.861755, 35.84851 ,
         35.838802, 35.82086 , 35.799923, 35.773006, 35.75073 ,
         35.730766, 35.714836, 35.6961  , 35.671745, 35.650074,
...
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan,       nan,       nan,       nan,       nan,
               nan]]], dtype=float32)
Coordinates:
  * depth    (depth) float32 0.494 1.541 2.646 ... 4.833e+03 5.275e+03 5.728e+03
  * time     (time) datetime64[ns] 2015-01-01T12:00:00 ... 2015-01-31T12:00:00
    lon      (location) float64 -217.0 -216.9 -216.9 ... -210.1 -210.1 -210.0
    lat      (location) float64 -38.95 -38.95 -38.95 ... -38.95 -38.95 -38.95
Dimensions without coordinates: location
Attributes:
    regrid_method:  bilinear

In [118]:
north[["so","zos"]]

<xarray.Dataset>
Dimensions:    (time: 31, depth: 50, latitude: 8, longitude: 85)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * latitude   (latitude) float32 -39.25 -39.17 -39.08 ... -38.83 -38.75 -38.67
  * time       (time) float64 5.698e+05 5.698e+05 ... 5.705e+05 5.705e+05
  * longitude  (longitude) float32 143.0 143.1 143.2 143.2 ... 149.8 149.9 150.0
Data variables:
    so         (time, depth, latitude, longitude) float32 ...
    zos        (time, latitude, longitude) float32 ...
Attributes: (12/25)
    title:                              daily mean fields from Global Ocean P...
    easting:                            longitude
    northing:                           latitude
    history:                            2022/05/25 21:54:07 MERCATOR OCEAN Ne...
    source:                             MERCATOR GLORYS12V1
    institution:                        MERCATOR OCEAN
    ...                                 ...
    FROM_ORIGINAL_FILE__longitude_max:  179.91667
    FROM_ORIGINAL_FILE__latitude_min:   -80.0
    FROM_ORIGINAL_FILE__latitude_max:   90.0
    z_min:                              0.494025
    z_max:                              5727.917
    _CoordSysBuilder:                   ucar.nc2.dataset.conv.CF1Convention

In [102]:
out


<xarray.Dataset>
Dimensions:  (time: 31, depth: 50, location: 141)
Coordinates:
  * depth    (depth) float32 0.494 1.541 2.646 ... 4.833e+03 5.275e+03 5.728e+03
  * time     (time) datetime64[ns] 2015-01-01T12:00:00 ... 2015-01-31T12:00:00
    lon      (location) float64 -217.0 -216.9 -216.9 ... -210.1 -210.1 -210.0
    lat      (location) float64 -38.95 -38.95 -38.95 ... -38.95 -38.95 -38.95
Dimensions without coordinates: location
Data variables:
    uo       (time, depth, location) float32 -0.00232 0.002073 ... nan nan
    vo       (time, depth, location) float32 -0.05847 -0.0439 ... nan nan
    so       (time, depth, location) float32 35.7 35.7 35.71 ... nan nan nan
    thetao   (time, depth, location) float32 17.32 17.33 17.35 ... nan nan nan
Attributes:
    regrid_method:  bilinear

In [101]:
raw.isel(time = 4,lat = 4)


<xarray.DataArray 'so' (depth: 50, lon: 85)>
array([[35.715813, 35.72802 , 35.7387  , ..., 35.60442 , 35.598316, 35.58153 ],
       [35.714287, 35.726494, 35.737175, ..., 35.60442 , 35.59679 , 35.580006],
       [35.714287, 35.726494, 35.737175, ..., 35.602894, 35.59679 , 35.580006],
       ...,
       [      nan,       nan,       nan, ...,       nan,       nan,       nan],
       [      nan,       nan,       nan, ...,       nan,       nan,       nan],
       [      nan,       nan,       nan, ...,       nan,       nan,       nan]],
      dtype=float32)
Coordinates:
  * depth    (depth) float32 0.494 1.541 2.646 ... 4.833e+03 5.275e+03 5.728e+03
    lat      float32 -38.92
    time     datetime64[ns] 2015-01-05T12:00:00
  * lon      (lon) float32 143.0 143.1 143.2 143.2 ... 149.8 149.8 149.9 150.0
Attributes:
    long_name:      Salinity
    standard_name:  sea_water_salinity
    units:          1e-3
    unit_long:      Practical Salinity Unit
    cell_methods:   area: mean
    _ChunkSizes:    [  1   7 341 720]

In [99]:
raw

<xarray.DataArray 'so' (time: 31, depth: 50, lat: 8, lon: 85)>
array([[[[35.676136, ..., 35.598316],
         ...,
         [35.73565 , ..., 35.595264]],

        ...,

        [[      nan, ...,       nan],
         ...,
         [      nan, ...,       nan]]],


       ...,


       [[[35.763115, ..., 35.52507 ],
         ...,
         [35.68529 , ..., 35.57085 ]],

        ...,

        [[      nan, ...,       nan],
         ...,
         [      nan, ...,       nan]]]], dtype=float32)
Coordinates:
  * depth    (depth) float32 0.494 1.541 2.646 ... 4.833e+03 5.275e+03 5.728e+03
  * lat      (lat) float32 -39.25 -39.17 -39.08 -39.0 ... -38.83 -38.75 -38.67
  * time     (time) datetime64[ns] 2015-01-01T12:00:00 ... 2015-01-31T12:00:00
  * lon      (lon) float32 143.0 143.1 143.2 143.2 ... 149.8 149.8 149.9 150.0
Attributes:
    long_name:      Salinity
    standard_name:  sea_water_salinity
    units:          1e-3
    unit_long:      Practical Salinity Unit
    cell_methods:   area: mean
    _ChunkSizes:    [  1   7 341 720]

In [93]:
raw

<xarray.DataArray 'so' (time: 31, depth: 50, lat: 8, lon: 85)>
array([[[[35.676136, ..., 35.598316],
         ...,
         [35.73565 , ..., 35.595264]],

        ...,

        [[      nan, ...,       nan],
         ...,
         [      nan, ...,       nan]]],


       ...,


       [[[35.763115, ..., 35.52507 ],
         ...,
         [35.68529 , ..., 35.57085 ]],

        ...,

        [[      nan, ...,       nan],
         ...,
         [      nan, ...,       nan]]]], dtype=float32)
Coordinates:
  * depth    (depth) float32 0.494 1.541 2.646 ... 4.833e+03 5.275e+03 5.728e+03
  * lat      (lat) float32 -39.25 -39.17 -39.08 -39.0 ... -38.83 -38.75 -38.67
  * time     (time) datetime64[ns] 2015-01-01T12:00:00 ... 2015-01-31T12:00:00
  * lon      (lon) float32 143.0 143.1 143.2 143.2 ... 149.8 149.8 149.9 150.0
Attributes:
    long_name:      Salinity
    standard_name:  sea_water_salinity
    units:          1e-3
    unit_long:      Practical Salinity Unit
    cell_methods:   area: mean
    _ChunkSizes:    [  1   7 341 720]